In [1]:
import qiskit
from qiskit.providers.aer.noise import NoiseModel
from qiskit import IBMQ, Aer
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
def cnx(qc, *qubits):
    if len(qubits) >= 3:
        last = qubits[-1]
        # A matrix: (made up of a  and Y rotation, lemma4.3)
        qc.crz(np.pi/2, qubits[-2], qubits[-1])
        qc.cu3(np.pi/2, 0, 0, qubits[-2],qubits[-1])
        
        # Control not gate
        cnx(qc,*qubits[:-2],qubits[-1])
        
        # B matrix (pposite angle)
        qc.cu3(-np.pi/2, 0, 0, qubits[-2], qubits[-1])
        
        # Control
        cnx(qc,*qubits[:-2],qubits[-1])
        
        # C matrix (final rotation)
        qc.crz(-np.pi/2,qubits[-2],qubits[-1])
    elif len(qubits)==3:
        qc.ccx(*qubits)
    elif len(qubits)==2:
        qc.cx(*qubits)

In [3]:
import numpy as np
from qiskit import IBMQ, QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import plot_histogram, plot_state_city

In [4]:
provider=IBMQ.load_account()

C:\Users\USER\anaconda3\envs\quantum\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [5]:
def increment_gate(qwc, q, subnode,n):
    for i in range(n):
        cnx(qwc, subnode, q[i:])
        cnx(qwc, subnode, q[i:])
        cnx(qwc, subnode, q[i:])
    qwc.barrier()
    return qwc

def decrement_gate(qwc, q, subnode,n):
    qwc.x(subnode)
    
    for i in range(1,n):
        qwc.x(q[i])
    
    for i in range(n-1):
        cnx(qwc, subnode, q[i:])
        qwc.x(q[i+1])

    cnx(qwc, subnode[0], q[n-1])
    qwc.x(subnode[0])
    return qwc
  
def ibmsim(circ):
    ibmqBE = IBMQ.load_account().get_backend('ibmq_16_melbourne')
    noise_model = NoiseModel.from_backend(ibmqBE)
    return execute(circ,Aer.get_backend('qasm_simulator'), shots=1000,noise_model=noise_model).result().get_counts(circ)

In [6]:
def runQWC(n,vec,times):
    qnodes = QuantumRegister(n,'qc')
    qsubnodes = QuantumRegister(1,'qanc')
    csubnodes = ClassicalRegister(1,'canc')
    cnodes = ClassicalRegister(n,'cr')

    qwc = QuantumCircuit(qnodes, qsubnodes, cnodes, csubnodes)
    qwc.initialize(vec,qnodes)
    
    for i in range(times):
        qwc.h(qsubnodes[0])
        increment_gate(qwc, qnodes, qsubnodes,n)
        decrement_gate(qwc,qnodes,qsubnodes,n)
        qwc.measure(qnodes, cnodes)
    return qwc

In [7]:
binnQ = lambda x,n : ''.join(reversed( [str((x >> i) & 1) for i in range(n)] ) )

In [8]:
def convert(binary_string):
    vec=np.zeros(2**len(binary_string))
    vec[int(binary_string,2)]=1
    return vec

In [9]:
string="AliShaib"
asc=[]
for c in string:
    asc.append(binnQ(ord(c),7))

In [ ]:
import random
waves=[]
colors=[]

for c in asc:
    step = 20
    qwc = runQWC(7,convert(c),step)
    waves.append(ibmsim(qwc))
    
    step = random.randint(1,50)
    qwc = runQWC(2,convert('00'),step)
    colors.append(ibmsim(qwc))

ibmqfactory.load_account:WARNING:2021-04-11 16:16:50,023: Credentials are already in use. The existing account in the session will be replaced.
C:\Users\USER\anaconda3\envs\quantum\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2021-04-11 16:17:03,577: Credentials are already in use. The existing account in the session will be replaced.
ibmqfactory.load_account:WARNING:2021-04-11 16:17:19,737: Credentials are already in use. The existing account in the session will be replaced.
ibmqfactory.load_account:WARNING:2021-04-11 16:17:36,843: Credentials are already in use. The existing account in the session will be replaced.
ibmqfactory.load_account:WARNING:2021-04-11 16:17:49,525: Credentials are already in use. The existing account in the session will 

In [ ]:
def convert_dicts_to_array(dicts,n):
    counts=np.zeros((len(dicts),2**n))

    for i in range(len(dicts)):
        for c in dicts[i]:
            counts[i,int(c[2:],2)]=dicts[i][c]
        counts[i,:]=counts[i]/np.sum(counts[i])
    return counts

In [ ]:
waves_arr=convert_dicts_to_array(waves,7)
colors_arr=convert_dicts_to_array(colors,2)

In [ ]:
for i in range(waves_arr.shape[0]):
    plt.plot(waves_arr[i],c=colors_arr[i,:3])